In [3]:
%matplotlib ipympl
from QDP import qdp, atom
import os.path
import numpy as np
from scipy.stats import poisson
from scipy import optimize
import matplotlib.pyplot as plt
dp = '/home/ebert/jupyter-notebooks/FNode-data'

exclude = [2]

In [93]:
exp_name = '2018_07_19_22_48_23_csrb-hd-odt-heating-1000ms'
# exp_name = ''
if exp_name:
    exp_date = '_'.join(exp_name.split('_')[:3])
    experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
else:
    experiment_file = ''
q = qdp.QDP(base_data_path=dp)
q.load_data_file(experiment_file)
print "ivars: {}".format(q.experiments[0]['variable_list'])
roi_map = ['cs', 'rb']
cs_cuts = [[27],[2]]
rb_cuts = [[30],[3]]
marker_styles = [
    dict(color='cornflowerblue', marker='o', markeredgecolor='b', linestyle='None'),
    dict(color='indianred', marker='d', markeredgecolor='r', linestyle='None')
]
q.set_thresholds(cs_cuts, roi=0)
q.set_thresholds(rb_cuts, roi=1)
q.generate_thresholds(loss=False)
retention = q.apply_thresholds(exclude_rois=exclude)
# for l in ['retention', 'ivar', 'error']:
#     retention[l] = np.delete(retention[l],-6,1)
tbl_str = "ivar:\t{0}\tretention:{1} +- {2}"
entries = np.product(retention['retention'].shape[:-1])
for i in range(entries):
    print(tbl_str.format(
        retention['ivar'].reshape((entries,-1))[i][0],
        retention['retention'].reshape((entries,-1))[i],
        retention['error'].reshape((entries,-1))[i]
    ))
# q.save_experiment_data()
q.experiments[0]['variable_list']
try:
    xlab = q.experiments[0]['variable_desc'][q.experiments[0]['variable_list'][0]]['description']
except IndexError:
    print("Only one iteration")
    xlab = ''

fig, ax = plt.subplots()
for e in range(len(retention['ivar'])):
    for r in range(2):
        ax.errorbar(
            retention['ivar'][e, :, r], retention['retention'][e, :, r],
            yerr=retention['error'][e, :, r],
            **marker_styles[r]
        )
ax.set_ylim(0, 1.01)
ax.set_xlabel(xlab)
ax.set_ylabel('F=4(2) pop.')

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'retention_scan.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

data at: 2018_07_19/2018_07_19_22_48_23_csrb-hd-odt-heating-1000ms/results.hdf5
name 'fort_exp' is not defined
name 'rb_uwave_freq' is not defined
ivars: [u'fort_drop_us']
--------------------
--------------------
--------------------
--------------------
On entry to DGESDD parameter number 5 had an illegal value
There may be some issue with your guess: `[  1.39650695e-17   0.00000000e+00   0.00000000e+00]`
--------------------
On entry to DGESDD parameter number 5 had an illegal value
There may be some issue with your guess: `[  1.39650695e-17   0.00000000e+00   0.00000000e+00]`
--------------------
ivar:	0.0	retention:[ 0.88741722  0.83425414         nan] +- [ 0.02324488  0.02324488  0.02324488]
ivar:	2.5	retention:[ 0.84666667  0.87562189         nan] +- [ 0.01957619  0.01957619  0.01957619]
ivar:	5.0	retention:[ 0.80821918  0.84251969         inf] +- [ 0.02718299  0.02718299  0.02718299]
ivar:	7.5	retention:[ 0.7972973   0.73493976         nan] +- [ 0.0407432  0.0407432  0.0407432]

FigureCanvasNbAgg()

In [119]:
def fnode_release_recapture(t, T_uk, a='cs'):
    wr_um = 2.3
    zr_um = 15.6  # calculated
    pfort_nom = 80.  # 80 mV on detector
    pfort_act = 135.6  # mV on detector
    fort_depth_fac = pfort_act/pfort_nom
    if a == 'cs':
        U_mk = 1.38*fort_depth_fac
        fr_khz = 37.4*np.sqrt(fort_depth_fac)
        fa_khz = 3.730*np.sqrt(fort_depth_fac)
        m_au = 132.905
    else:  # rb
        U_mk = 0.81*fort_depth_fac
        fr_khz = 35.4*np.sqrt(fort_depth_fac)
        fa_khz = 3.57*np.sqrt(fort_depth_fac)
        m_au = 86.909
    n=50000
    try:
        res = np.zeros(len(t))
        for i in xrange(len(t)):
            res[i] = atom.release_recapture(t[i], T_uk, U_mk, wr_um, zr_um, fr_khz, fa_khz, n=n, m_au=m_au)
        return res
    except Exception as e:
        print(e)
        return atom.release_recapture(t, T_uk, U_mk, wr_um, zr_um, fr_khz, fa_khz, n=n, m_au=m_au)

def residuals(T_uk, y, t, f0, a='cs'):
    res = y - f0*fnode_release_recapture(t, T_uk, a=a)
    #print "T_uk = {}, -> {}".format(T_uk, np.sum(np.power(res, 2)))
    return res

def ls_err(T_uk, y, t, f0, w, a='cs'):
    res = residuals(T_uk, y, t, f0, a=a)
    return np.dot(np.power(res, 2), w)

In [120]:
p0_opt = [retention['retention'][e, 0, 0],retention['retention'][e, 0, 1]]

# Minimize along T axis with initial retention guess

In [122]:
# do the fit by hand because curve_fit and least_squares dont seem to want to take large enough step sizes when searching
# this is a problem when the function is not very repeatable, like in a monte carlo
e=0
ls_data = []
start = [40, 40]
end = [200, 200]
n = 11
for r in range(2):
    ls_data.append(np.array([
        [T, ls_err(T, retention['retention'][e, :, r], retention['ivar'][e, :, r], retention['retention'][e, 0, r], np.power(retention['error'][e, :, r], -0.5), a=roi_map[r])] 
        for T in np.linspace(start[r],end[r],n)
    ]))

In [123]:
fig, ax = plt.subplots()
# print ls_data
z = []
T_opt = []
for r in range(2):
    xs = np.linspace(start[r],end[r],n)
    z.append(np.polyfit(ls_data[r][:,0], ls_data[r][:,1], 2))
    ax.plot(ls_data[r][:,0], ls_data[r][:,1], ['bo', 'rx'][r])
    ax.plot(xs, np.poly1d(z[r])(xs))
    T_opt.append(optimize.fmin(np.poly1d(z[r]), 63)[0])

FigureCanvasNbAgg()

Optimization terminated successfully.
         Current function value: 0.123194
         Iterations: 22
         Function evaluations: 44
Optimization terminated successfully.
         Current function value: 0.254804
         Iterations: 21
         Function evaluations: 42


# Minimize along p0 axis with T_opt

In [137]:
# do the fit by hand because curve_fit and least_squares dont seem to want to take large enough step sizes when searching
# this is a problem when the function is not very repeatable, like in a monte carlo
e=0
ls_data = []
start = np.array(p0_opt) - 0.0125
end = np.array(p0_opt) + 0.0125
n = 15
for r in range(2):
    ls_data.append(np.array([
        [p0, ls_err(T_opt[r], retention['retention'][e, :, r], retention['ivar'][e, :, r], p0, np.power(retention['error'][e, :, r], -0.5), a=roi_map[r])] 
        for p0 in np.linspace(start[r],end[r],n)
    ]))

In [138]:
fig, ax = plt.subplots()
# print ls_data
z = []
p0_opt = []
for r in range(2):
    xs = np.linspace(start[r],end[r],n)
    z.append(np.polyfit(ls_data[r][:,0], ls_data[r][:,1], 2))
    ax.plot(ls_data[r][:,0], ls_data[r][:,1], ['bo', 'rx'][r])
    ax.plot(xs, np.poly1d(z[r])(xs))
    p0_opt.append(min(optimize.fmin(np.poly1d(z[r]), 63)[0], 1))

FigureCanvasNbAgg()

Optimization terminated successfully.
         Current function value: 0.058770
         Iterations: 23
         Function evaluations: 46
Optimization terminated successfully.
         Current function value: 0.132657
         Iterations: 23
         Function evaluations: 46


# Minimize along T axis with p0_opt

In [139]:
# do the fit by hand because curve_fit and least_squares dont seem to want to take large enough step sizes when searching
# this is a problem when the function is not very repeatable, like in a monte carlo
e=0
ls_data = []
start = np.array(T_opt) - 15
end = np.array(T_opt) + 15
n = 21
for r in range(2):
    ls_data.append(np.array([
        [T, ls_err(T, retention['retention'][e, :, r], retention['ivar'][e, :, r], min(p0_opt[r], 1), np.power(retention['error'][e, :, r], -0.5), a=roi_map[r])] 
        for T in np.linspace(start[r],end[r],n)
    ]))

In [140]:
fig, ax = plt.subplots()
# print ls_data
z = []
T_opt = []
for r in range(2):
    xs = np.linspace(start[r],end[r],n)
    z.append(np.polyfit(ls_data[r][:,0], ls_data[r][:,1], 2, cov=True))
    ax.plot(ls_data[r][:,0], ls_data[r][:,1], ['bo', 'rx'][r])
    ax.plot(xs, np.poly1d(z[r][0])(xs))
    T_opt.append(-z[r][0][1]/(2*z[r][0][0]))

FigureCanvasNbAgg()

# plot final fit

In [43]:
def quad_uncert(popt, pcov):
    return np.sqrt(np.abs(pcov[0][0])*np.power(popt[1]/(2.*np.power(popt[0],2)), 2) + np.abs(pcov[1][1])*np.power(1./(2.*popt[0]), 2))

In [44]:
quad_uncert(z[1][0], z[1][1])

7.6123498076146934

In [45]:
np.diagonal(np.abs(z[0][1]))[:2]

array([  4.55883126e-12,   1.66834504e-07])

In [46]:
z[0][0][1]/(2.*z[0][0][0]**2)

-2824090.9613695936

In [47]:
z[0][0][0]**2

1.1420278425082057e-09

In [143]:
fig, ax = plt.subplots()
for e in range(len(retention['ivar'])):
    for r in range(2):
        ax.errorbar(
            retention['ivar'][e, :, r], retention['retention'][e, :, r],
            yerr=retention['error'][e, :, r],
            **marker_styles[r]
        )
        xs = np.linspace(0, max(retention['ivar'][e,:,r]), 30)
        ax.plot(xs, fnode_release_recapture(xs, T_opt[r], a=roi_map[r])*p0_opt[r], 'k-')
        ax.text(2.55, 0.25-0.05*r, "T_{} = {:.1f} uK".format(roi_map[r], T_opt[r]))
    
ax.set_ylim(0, 1.01)
ax.set_xlabel(xlab)
ax.set_ylabel('Retention')
fmts = ['pdf', 'png']
fn = os.path.join(dp, os.path.dirname(experiment_file), 'temperature_fit.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

# Find maximally sensitive drop time

In [74]:
# where can I be most sensitive for a temperature measurement?
fig, ax = plt.subplots()
for r in range(2):
    sensitivity = []
    c = [167, 96][r]
    dropts = np.linspace(5, 60, 31)
    for dropt in dropts:
        ts = np.linspace(c-10, c+10, 5)
        tdata = [fnode_release_recapture(dropt, T, a=roi_map[r]) for T in ts]
        sensitivity.append(np.polyfit(ts, tdata, 1)[0]*1000)
    ax.plot(dropts, sensitivity, 'br'[r]+'-')
    
# ax.set_ylim(0, 1.01)
ax.set_xlabel('drop time (ms)')
ax.set_ylabel('d retention/dT a.u.')

FigureCanvasNbAgg()

object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' h

Text(0,0.5,u'd retention/dT a.u.')